In [ ]:
def list_vectors_fun(A):
    n = len(A)
    if n == 5:
        for a0 in A[0]:
            for a1 in A[1]:
                for a2 in A[2]:
                    for a3 in A[3]:
                        for a4 in A[4]:
                            yield tuple({a0,a1,a2,a3,a4})
    elif n == 4:
        for a0 in A[0]:
            for a1 in A[1]:
                for a2 in A[2]:
                    for a3 in A[3]:
                        yield tuple({a0,a1,a2,a3})
    elif n == 3:
        for a0 in A[0]:
            for a1 in A[1]:
                for a2 in A[2]:
                    yield tuple({a0,a1,a2})
    elif n == 6:
        for a0 in A[0]:
            for a1 in A[1]:
                for a2 in A[2]:
                    for a3 in A[3]:
                        for a4 in A[4]:
                            for a5 in A[5]:
                                yield tuple({a0,a1,a2,a3,a4,a5})
    elif n == 7:
        for a0 in A[0]:
            for a1 in A[1]:
                for a2 in A[2]:
                    for a3 in A[3]:
                        for a4 in A[4]:
                            for a5 in A[5]:
                                for a6 in A[6]:
                                    yield tuple({a0,a1,a2,a3,a4,a5,a6})
    eliif n == 2:
        for a0 in A[0]:
            for a1 in A[1]:
                yield tuple({a0,a1})

In [ ]:
import time 
H5 = list(graphs(5)) 
G5 = [] #list of connected graphs with 5 vertices
for i in range(len(H5)):
    if H5[i].is_connected():
        G5.append(H5[i])
###The following is the usual entry-wise partial order to compare lists (vectors).###
class T(list):
    def __le__(left, right):
        return all(l <= r for l, r in zip(left, right))
###Next function returns all the non-negative vectors that are less or equal to a given vector (usual partial order)###
def IntegerVectorsIterator(vect, min = None):
    if len(vect) == 0:
        yield []
    else:
        if min is None:
            min = [0]*len(vect)
        if vect < min:
            return
        else:
            for vec in IntegerVectorsIterator(vect[1:], min = min[1:]):
                yield [min[0]]+vec
            for j in range(min[0]+1,vect[0]+1):
                for vec in IntegerVectorsIterator(vect[1:]):
                    yield [j]+vec
###Given a vector, next function sums 1 to a given position of the vector.###
def up(vector,pos):
    return vector[:pos]+[vector[pos]+1]+vector[pos+1:]
###The following function returns the minimal elements of a set of integer vectors.###
def min_elems_integers(Set):
    if len(Set) == 0:
        return []
    else:
        return [v for v in Set if not any(T(w) <= T(v) and v!=w for w in Set ) ]
###Next funtion takes as input a set of vectors, and the output is the minimal vector greater than all the input vectors.###
def up_vector_tuple(List,n): #list of tuples
    m = len(List)
    if 2 <= m:
        return tuple( max(List[k][i] for k in range(m)) for i in range(n) )
    else:
        return List[0]
###Reduced matrix, that is, a submatrix resulting from erasing row and columns indexed by s.###
def reduced_matrix(L,s): #Input: square matrix and 1<=s<=n
    n = L.nrows()
    I = [0..(n-1)]
    J = I[:s] + I[s+1:]
    return L.matrix_from_rows_and_columns(J,J)
###Step (3).###
def list_vectors_ite(A):
    na = len(A)
    k = list( len(A[s])-1 for s in range(na) )
    for ind in IntegerVectorsIterator(k):
        yield tuple({A[s][ind[s]] for s in range(na)})
###Next functions does the same as min_elems_integers() but as an iterator to save memory.###
def minimals_iterator(Alpha):
    for v in Alpha:
        if not any(T(w) <= T(v) and v != w for w in Alpha):
            yield v
###Step (13):###
###Next function computes all minimal vectors such that we get an almost non-sing. M-matrix from a quasi non-sing. M-matrix.###
def min_calC(L,d,Alpha,search_neg): #From a quasi non-singular M-matrix with Diagonal d, C(Diag(d)-L=M), det(M) can be negative, or zero or positive already
    n = L.nrows()
    M = diagonal_matrix(d) - L
    zero_vec = [0]*n #list(0 for i in range(n))
    Mdet = M.det()
    if 0 <= Mdet:
        return [[zero_vec],search_neg]
    else:
        C = []
        sn_new = []
        coeff_x = list(reduced_matrix(M,q).det() for q in range(n))
        maxd = list(ceil((-Mdet) / coeff_x[s]) for s in range(n))
        for dnew in IntegerVectorsIterator(maxd):
            if any(T(c) <= T(dnew) for c in C):
                continue
            else:
                dd = list(dnew[t] + d[t] for t in range(n))
                if dd not in search_neg:
                    newM = diagonal_matrix(dnew) + M
                    newMdet = newM.det()
                    if 0 <= newMdet:
                        C.append(dnew)
                    elif all((not T(snn) <= T(dd)) for snn in sn_new):
                        sn_new.append(dd)
        search_neg_new = set( tuple(ele) for ele in search_neg+sn_new)
        search_neg_new = list(list(ele) for ele in search_neg_new )
        if C == []:
            return [ [zero_vec],search_neg_new]
        else:
            return [C,search_neg_new]
###Function at step (14)###
def Add_special(d,D): #adds d to list D if minimal and erase every element above it.
    if len(D) == 0:
        return [d]
    elif d in D:
        return D
    elif any(T(dd) <= T(d) for dd in D):
        return D
    else:
        DD = [dd for dd in D if not T(d) <= T(dd)] + [d]
        return DD
###Function at step (14) (recursive; taking a list of vectors as input)###
def Add_special_list(List,D): #adds d to list D if minimal and erase every element above it.
    if len(List) == 0:
        return D
    elif len(List) == 1:
        return Add_special(List[0],D)
    else:
        return Add_special(List[-1], Add_special_list(List[:-1],D) )
###The following is the funcion at step (6).###
def Find(L,d,S,D,Alpha,search_neg):
    k = len(S)
    n = L.nrows()
    if k == 0:
        L_d = diagonal_matrix(d) - L
        determinant = L_d.det()
        if 0 <= determinant:
            D = Add_special(d,D)
            return [D,search_neg]
        else:
            if d in search_neg:
                return [D,search_neg]
            else:
                [quasi_almost,search_neg] = min_calC(L,d,Alpha,search_neg)
                new_d = list(list(d_prime[j] + d[j] for j in range(n)) for d_prime in quasi_almost)
                D = Add_special_list(new_d,D)
                return [D,search_neg]
    elif k == 1:
        for i in range(n):
            if i not in S:
                [D,search_neg] = Find(L,up(d,i),list(),D,Alpha,search_neg)
        return [D,search_neg]
    else:
        if k < n:
            for i in range(n):
                if i not in S:
                    [D,search_neg] = Find(L,up(d,i),list(),D,Alpha,search_neg)
        for pair in Combinations(S,2):
            [D,search_neg] = Find(L,up(up(d,pair[0]),pair[1]),list(),D,Alpha,search_neg)
        return [D,search_neg]
###Main algorithm. Input: square integer matrix L of size with non-negative entries and zero-diagonal (we assume the Input is correct and that it is a matrix of size at least 2).###
###Output: $\min D_{\geq 0}(L)$.###
def minD_0(L):
    A_temp = {}
    A = []
    D = []
    n = L.nrows() #size of L
    if n == 2:
        constant_coeff = L.det() 
        if -1 <= constant_coeff:
            return [[1,1]]
        else:
            cc = (-1)*constant_coeff + 1
            D = list([i,ceil( (cc-1)/i )] for i in range(1,cc))
            return min_elems_integers(D)
    else:
        if not Graph(L, format = 'adjacency_matrix').is_connected(): #L reducible
            for s in range(n):
                if sum(L.column(s)) == 0:
                    L_s = reduced_matrix(L,s)
                    D_1 = list( ds[:s] + [1] + ds[s:] for ds in minD_0(L_s) if (diagonal_matrix(ds)-L_s).det() > 0)
                    D_2 = set(tuple(up(ds,i)[:s] + [1] + up(ds,i)[s:]) for i in range(n-1) for ds in minD_0(L_s) if (diagonal_matrix(ds)-L_s).det() == 0)
                    D_2 = list(list(d2) for d2 in D_2)
                    return min_elems_integers(D_1 + D_2)
            for s in range(n):
                L_s = reduced_matrix(L,s)
                A_temp[s] = minD_0(L_s)  #step(1)
                A.append(list( tuple(d_temp[:s]+[1]+d_temp[s:]) for d_temp in A_temp[s] )) #step(2)
            Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
            Alpha = list( list(element) for element in Alpha)
            search_neg =[]
            for d in minimals_iterator(Alpha): #...(3) "and" (4)
                L_d = diagonal_matrix(d) - L
                S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
                [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
            return D #step (15)
        else:
            canonical_as = list( sum(L.row(i)) for i in range(n) )
            one_vec = [1]*n
            if T(one_vec) <= T(canonical_as):
                D.append(canonical_as)
            else: #there is a zero row
                for s in range(n):
                    if sum(L.row(s)) == 0:
                        L_s = reduced_matrix(L,s)
                        D_1 = list( ds[:s] + [1] + ds[s:] for ds in minD_0(L_s) if (diagonal_matrix(ds)-L_s).det() > 0)
                        D_2 = set(tuple(up(ds,i)[:s] + [1] + up(ds,i)[s:]) for i in range(n-1) for ds in minD_0(L_s) if (diagonal_matrix(ds)-L_s).det() == 0)
                        D_2 = list(list(d2) for d2 in D_2)
                        return min_elems_integers(D_1 + D_2)
            for s in range(n):
                L_s = reduced_matrix(L,s)
                A_temp[s] = minD_0(L_s)  #step(1)
                A.append(list( tuple(d_temp[:s]+[1]+d_temp[s:]) for d_temp in A_temp[s] )) #step(2)
            Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
            Alpha = list( list(element) for element in Alpha)
            search_neg = []
            for d in minimals_iterator(Alpha): #...(3) "and" (4)
                L_d = diagonal_matrix(d) - L
                S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
                [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
            return D #step (15)
###We can use minD_0(L) to compute the d-arithmetical structures of L. We assume L to be irreducible, i.e., finite (and non-empty) set of A.S's.###
def d_arithmetical_structures(D,L):
    E = []
    for d in D:
        A = diagonal_matrix(d) - L
        if A.det() == 0:
            E.append(d)
    return E
mD = {}; DAS = {}
                
#fast anyway (or twins didn't really improve the running time): 0,1,3,9    
#twin-free graphs: 4 (fast),8 (~3 minutes),11 (fast),12 ,13
print('start 0')
s0 = time.time()
def minD_0_0(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        A.append(list(set(list( tuple([max(d_temp[i] for i in range(3))]+d_temp) for d_temp in A_temp[s] )+list( tuple([d_temp[0]]+[max(d_temp[i] for i in range(3))]+d_temp[1:]) for d_temp in A_temp[s] ) + list( tuple(d_temp[:2] + [max(d_temp[i] for i in range(3))]+d_temp[2:] ) for d_temp in A_temp[s] )+list( tuple(d_temp[:3] + [max(d_temp[i] for i in range(3))]+d_temp[3:] ) for d_temp in A_temp[s] ) )))
    #Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_fun_2(A)) )
    Alpha = A[0]
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[0].graph6_string())] = minD_0_0(G5[0].adjacency_matrix())
DAS[str(G5[0].graph6_string())] = []
for d in mD[str(G5[0].graph6_string())]:
    L = diagonal_matrix(d) - G5[0].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[0].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[0].graph6_string())]))
print(len(DAS[str(G5[0].graph6_string())]))
print(s1-s0)
print('end 0')

print('start 1')
s0 = time.time()
def minD_0_1(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,3,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s in [0,3,4]:
            A.append(list( tuple(d_temp[:s] + [1] + d_temp[s:] ) for d_temp in A_temp[s] ))
        elif s == 1:
            A.append(list( tuple(d_temp[:1] + [d_temp[1],d_temp[1]] + d_temp[2:]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[1].graph6_string())] = minD_0_1(G5[1].adjacency_matrix())
DAS[str(G5[1].graph6_string())] = []
for d in mD[str(G5[1].graph6_string())]:
    L = diagonal_matrix(d) - G5[1].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[1].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[1].graph6_string())]))
print(len(DAS[str(G5[1].graph6_string())]))
print(s1-s0)
print('end 1')

print('start 2')
s0 = time.time()
def minD_0_2(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,2,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s in [1,2,4]:
            A.append(list( tuple(d_temp[:s] + [1] + d_temp[s:] ) for d_temp in A_temp[s] ))
        elif s == 0:
            A.append(list( tuple([d_temp[2]] + d_temp) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[2].graph6_string())] = minD_0_2(G5[2].adjacency_matrix())
DAS[str(G5[2].graph6_string())] = []
for d in mD[str(G5[2].graph6_string())]:
    L = diagonal_matrix(d) - G5[2].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[2].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[2].graph6_string())]))
print(len(DAS[str(G5[2].graph6_string())]))
print(s1-s0)
print('end 2')

print('start 3')
s0 = time.time()
def minD_0_3(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,2,3,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s in [2,3,4]:
            A.append(list( tuple(d_temp[:s] + [1] + d_temp[s:] ) for d_temp in A_temp[s] ))
        elif s == 0:
            A.append(list( tuple([d_temp[0]] + d_temp) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[3].graph6_string())] = minD_0_3(G5[3].adjacency_matrix())
DAS[str(G5[3].graph6_string())] = []
for d in mD[str(G5[3].graph6_string())]:
    L = diagonal_matrix(d) - G5[3].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[3].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[3].graph6_string())]))
print(len(DAS[str(G5[3].graph6_string())]))
print(s1-s0)
print('end 3')


print('start 4')
s0 = time.time()
def minD_0_4(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,2,3,4]:#0,3 and 1,2
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(tuple([1]+dt) for dt in A_temp[s]))
            #A.append(list(set( list( tuple([dt[0],max(dt[0],dt[1]),max(dt[1],dt[2]),dt[2],dt[3]] ) for dt in A_temp[s] ) + list( tuple([dt[2],max(dt[0],dt[1]),max(dt[1],dt[0]),dt[2],dt[3]]) for dt in A_temp[s] ) )))
        elif s == 3:
            A.append(list(tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s]) )
        elif s == 1:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
            #A.append(list(set( list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) )))
        elif s == 2:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[4].graph6_string())] = minD_0_4(G5[4].adjacency_matrix())
DAS[str(G5[4].graph6_string())] = []
for d in mD[str(G5[4].graph6_string())]:
    L = diagonal_matrix(d) - G5[4].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[4].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[4].graph6_string())]))
print(len(DAS[str(G5[4].graph6_string())]))
print(s1-s0)
print('end 4')

print('start 5')
s0 = time.time()
def minD_0_5(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [1,2,3,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 1:
            A.append(list( tuple(d_temp[:s]+d_temp) for d_temp in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp[:s]+[1]+d_temp[s:]) for d_temp in A_temp[s] )) 
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[5].graph6_string())] = minD_0_5(G5[5].adjacency_matrix())
DAS[str(G5[5].graph6_string())] = []
for d in mD[str(G5[5].graph6_string())]:
    L = diagonal_matrix(d) - G5[5].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[5].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[5].graph6_string())]))
print(len(DAS[str(G5[5].graph6_string())]))
print(s1-s0)
print('end 5')

print('start 6')
s0 = time.time()
def minD_0_6(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,3]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(set(list( tuple([max(d_temp[i] for i in range(2))]+d_temp) for d_temp in A_temp[s] )+list( tuple([max(d_temp[i] for i in range(2))]+[d_temp[1]]+[d_temp[0]]+d_temp[2:]) for d_temp in A_temp[s] ) + list( tuple(d_temp[:1] + [max(d_temp[i] for i in range(2))]+d_temp[1:] ) for d_temp in A_temp[s] )+list( tuple([d_temp[1]] + [max(d_temp[i] for i in range(2))]+[d_temp[0]]+d_temp[2:] ) for d_temp in A_temp[s] ) + list( tuple(d_temp[:2] + [max(d_temp[i] for i in range(2))]+d_temp[2:] ) for d_temp in A_temp[s] )+list( tuple([d_temp[1],d_temp[0]] + [max(d_temp[i] for i in range(2))]+d_temp[2:] ) for d_temp in A_temp[s] ) )))
        elif s == 3:
            A.append(list( tuple(d_temp + d_temp[s:]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[6].graph6_string())] = minD_0_6(G5[6].adjacency_matrix())
DAS[str(G5[6].graph6_string())] = []
for d in mD[str(G5[6].graph6_string())]:
    L = diagonal_matrix(d) - G5[6].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[6].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[6].graph6_string())]))
print(len(DAS[str(G5[6].graph6_string())]))
print(s1-s0)
print('end 6')

print('start 7')
s0 = time.time()
def minD_0_7(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,3]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(set( list(tuple([max(d_temp[i] for i in range(2))]+d_temp) for d_temp in A_temp[s] )+list(tuple([max(d_temp[i] for i in range(2))]+ [d_temp[1],d_temp[0]]+ d_temp[2:]) for d_temp in A_temp[s] ) + list( tuple(d_temp[:1] + [max(d_temp[i] for i in range(2))]+d_temp[1:] ) for d_temp in A_temp[s] )+list( tuple([d_temp[1]] + [max(d_temp[i] for i in range(2))]+[d_temp[0]]+ d_temp[2:] ) for d_temp in A_temp[s] ) + list( tuple(d_temp[:2] + [max(d_temp[i] for i in range(2))]+d_temp[2:] ) for d_temp in A_temp[s] )+list( tuple([d_temp[1],d_temp[0]] + [max(d_temp[i] for i in range(2))]+d_temp[2:] ) for d_temp in A_temp[s] ) )))
        elif s == 3:
            A.append(list( tuple(d_temp + d_temp[s:]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[7].graph6_string())] = minD_0_7(G5[7].adjacency_matrix())
DAS[str(G5[7].graph6_string())] = []
for d in mD[str(G5[7].graph6_string())]:
    L = diagonal_matrix(d) - G5[7].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[7].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[7].graph6_string())]))
print(len(DAS[str(G5[7].graph6_string())]))
print(s1-s0)
print('end 7')


print('start 8')
s0 = time.time()
def minD_0_8(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,2,3,4]:#0,3 and 1,2
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(tuple([1]+dt) for dt in A_temp[s]))
            #A.append(list(set( list( tuple([dt[0],max(dt[0],dt[1]),max(dt[1],dt[2]),dt[2],dt[3]] ) for dt in A_temp[s] ) + list( tuple([dt[2],max(dt[0],dt[1]),max(dt[1],dt[0]),dt[2],dt[3]]) for dt in A_temp[s] ) )))
        elif s == 3:
            A.append(list(tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s]) )
        elif s == 1:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
            #A.append(list(set( list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) )))
        elif s == 2:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[8].graph6_string())] = minD_0_8(G5[8].adjacency_matrix())
DAS[str(G5[8].graph6_string())] = []
for d in mD[str(G5[8].graph6_string())]:
    L = diagonal_matrix(d) - G5[8].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[8].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[8].graph6_string())]))
print(len(DAS[str(G5[8].graph6_string())]))
print(s1-s0)
print('end 8')

print('start 9')
s0 = time.time()
def minD_0_9(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,3,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s in [1,3,4]:
            A.append(list( tuple(d_temp[:s] + [1] + d_temp[s:] ) for d_temp in A_temp[s] ))
        elif s == 0:
            A.append(list( tuple([d_temp[1]] + d_temp) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[9].graph6_string())] = minD_0_9(G5[9].adjacency_matrix())
DAS[str(G5[9].graph6_string())] = []
for d in mD[str(G5[9].graph6_string())]:
    L = diagonal_matrix(d) - G5[9].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[9].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[9].graph6_string())]))
print(len(DAS[str(G5[9].graph6_string())]))
print(s1-s0)
print('end 9')


print('start 10')
s0 = time.time()
def minD_0_10(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list( tuple([d_temp[1]] + d_temp ) for d_temp in A_temp[s] ))
        elif s == 1:
            A.append(list( tuple(d_temp[:s] + [d_temp[2]] + d_temp[s:]) for d_temp in A_temp[s] ))
        elif s == 4:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ) )
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[10].graph6_string())] = minD_0_10(G5[10].adjacency_matrix())
DAS[str(G5[10].graph6_string())] = []
for d in mD[str(G5[10].graph6_string())]:
    L = diagonal_matrix(d) - G5[10].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[10].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[10].graph6_string())]))
print(len(DAS[str(G5[10].graph6_string())]))
print(s1-s0)
print('end 10')

print('start 11')
s0 = time.time()
def minD_0_11(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,2,3,4]:#0,3 and 1,2
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(tuple([1]+dt) for dt in A_temp[s]))
            #A.append(list(set( list( tuple([dt[0],max(dt[0],dt[1]),max(dt[1],dt[2]),dt[2],dt[3]] ) for dt in A_temp[s] ) + list( tuple([dt[2],max(dt[0],dt[1]),max(dt[1],dt[0]),dt[2],dt[3]]) for dt in A_temp[s] ) )))
        elif s == 3:
            A.append(list(tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s]) )
        elif s == 1:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
            #A.append(list(set( list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) )))
        elif s == 2:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[11].graph6_string())] = minD_0_11(G5[11].adjacency_matrix())
DAS[str(G5[11].graph6_string())] = []
for d in mD[str(G5[11].graph6_string())]:
    L = diagonal_matrix(d) - G5[11].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[11].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[11].graph6_string())]))
print(len(DAS[str(G5[11].graph6_string())]))
print(s1-s0)
print('end 11')

print('start 12')
s0 = time.time()
def minD_0_12(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,2,3,4]:#0,3 and 1,2
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(tuple([1]+dt) for dt in A_temp[s]))
            #A.append(list(set( list( tuple([dt[0],max(dt[0],dt[1]),max(dt[1],dt[2]),dt[2],dt[3]] ) for dt in A_temp[s] ) + list( tuple([dt[2],max(dt[0],dt[1]),max(dt[1],dt[0]),dt[2],dt[3]]) for dt in A_temp[s] ) )))
        elif s == 3:
            A.append(list(tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s]) )
        elif s == 1:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
            #A.append(list(set( list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) )))
        elif s == 2:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[12].graph6_string())] = minD_0_12(G5[12].adjacency_matrix())
DAS[str(G5[12].graph6_string())] = []
for d in mD[str(G5[12].graph6_string())]:
    L = diagonal_matrix(d) - G5[12].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[12].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[12].graph6_string())]))
print(len(DAS[str(G5[12].graph6_string())]))
print(s1-s0)
print('end 12')

print('start 13')
s0 = time.time()
def minD_0_13(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,2,3,4]:#0,3 and 1,2
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(tuple([1]+dt) for dt in A_temp[s]))
            #A.append(list(set( list( tuple([dt[0],max(dt[0],dt[1]),max(dt[1],dt[2]),dt[2],dt[3]] ) for dt in A_temp[s] ) + list( tuple([dt[2],max(dt[0],dt[1]),max(dt[1],dt[0]),dt[2],dt[3]]) for dt in A_temp[s] ) )))
        elif s == 3:
            A.append(list(tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s]) )
        elif s == 1:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
            #A.append(list(set( list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[0]),dt[3]]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[1]),dt[2],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) + list( tuple([max(dt[0],dt[2]),dt[1],dt[1],max(dt[2],dt[3]),max(dt[3],dt[0])]) for dt in A_temp[s] ) )))
        elif s == 2:
            A.append(list( tuple(dt[:s]+[1]+dt[s:]) for dt in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[13].graph6_string())] = minD_0_13(G5[13].adjacency_matrix())
DAS[str(G5[13].graph6_string())] = []
for d in mD[str(G5[13].graph6_string())]:
    L = diagonal_matrix(d) - G5[13].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[13].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[13].graph6_string())]))
print(len(DAS[str(G5[13].graph6_string())]))
print(s1-s0)
print('end 13')

print('start 14')
s0 = time.time()
def minD_0_14(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,3,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s in [1,3,4]:
            A.append(list( tuple(d_temp[:s] + [1] + d_temp[s:] ) for d_temp in A_temp[s] ))
        elif s == 0:
            A.append(list( tuple([d_temp[1]] + d_temp) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[14].graph6_string())] = minD_0_14(G5[14].adjacency_matrix())
DAS[str(G5[14].graph6_string())] = []
for d in mD[str(G5[14].graph6_string())]:
    L = diagonal_matrix(d) - G5[14].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[14].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[14].graph6_string())]))
print(len(DAS[str(G5[14].graph6_string())]))
print(s1-s0)
print('end 14')


print('start 15')
s0 = time.time()
def minD_0_15(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list(set( list( tuple([max(d_temp[i] for i in [1,2] )]+d_temp) for d_temp in A_temp[s] ) + list( tuple([d_temp[1]] + [d_temp[0]] + [max(d_temp[i] for i in [1,2] )]+ d_temp[2:] ) for d_temp in A_temp[s] )+ list( tuple([d_temp[1]] + [d_temp[0]] + [d_temp[2]] + [max(d_temp[i] for i in [1,2] )]+[d_temp[3]] ) for d_temp in A_temp[s] ) )))
        elif s == 1:
            A.append(list( tuple(d_temp[:s] + [1] + d_temp[s:]) for d_temp in A_temp[s] ))
        elif s == 4:
            A.append(list( tuple(d_temp[:s] + [1]) for d_temp in A_temp[s] ) )
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[15].graph6_string())] = minD_0_15(G5[15].adjacency_matrix())
DAS[str(G5[15].graph6_string())] = []
for d in mD[str(G5[15].graph6_string())]:
    L = diagonal_matrix(d) - G5[15].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[15].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[15].graph6_string())]))
print(len(DAS[str(G5[15].graph6_string())]))
print(s1-s0)
print('end 15')

print('start 16')
s0 = time.time()
def minD_0_16(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    print(n)
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,1,3]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 3:
            A.append(list( tuple(d_temp[:s]+d_temp[s:]+d_temp[s:]) for d_temp in A_temp[s] ))
        elif s ==1:
            A.append(list( tuple(d_temp[:s]+[1]+d_temp[s:]) for d_temp in A_temp[s] ))
        else:
            A.append(list( tuple([d_temp[1]]+d_temp) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[16].graph6_string())] = minD_0_16(G5[16].adjacency_matrix())
DAS[str(G5[16].graph6_string())] = []
for d in mD[str(G5[16].graph6_string())]:
    L = diagonal_matrix(d) - G5[16].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[16].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[16].graph6_string())]))
print(len(DAS[str(G5[16].graph6_string())]))
print(s1-s0)
print('end 16')

print('start 17')
s0 = time.time()
def minD_0_17(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    print(n)
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,2,3]:#0,1 and 2,4
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list( tuple(d_temp[:1]+d_temp) for d_temp in A_temp[s] ))
        elif s == 3:
            A.append(list( tuple(d_temp[:s]+[1]+d_temp[s:]) for d_temp in A_temp[s] ))
        else:
            A.append(list( tuple(d_temp[:s] + [d_temp[3]] + d_temp[s:]) for d_temp in A_temp[s] ))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[17].graph6_string())] = minD_0_17(G5[17].adjacency_matrix())
DAS[str(G5[17].graph6_string())] = []
for d in mD[str(G5[17].graph6_string())]:
    L = diagonal_matrix(d) - G5[17].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[17].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[17].graph6_string())]))
print(len(DAS[str(G5[17].graph6_string())]))
print(s1-s0)
print('end 17')

print('start 18')
s0 = time.time()
def minD_0_18(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0,2,4]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 0:
            A.append(list( tuple([d_temp[0]] + d_temp ) for d_temp in A_temp[s] ))
        elif s == 2:
            A.append(list( tuple(d_temp[:s] + [d_temp[2]] + d_temp[s:]) for d_temp in A_temp[s] ))
        elif s == 4:
            A.append(list( tuple(d_temp + [1]) for d_temp in A_temp[s] ) )
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[18].graph6_string())] = minD_0_18(G5[18].adjacency_matrix())
DAS[str(G5[18].graph6_string())] = []
for d in mD[str(G5[18].graph6_string())]:
    L = diagonal_matrix(d) - G5[18].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[18].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[18].graph6_string())]))
print(len(DAS[str(G5[18].graph6_string())]))
print(s1-s0)
print('end 18')

s0 = time.time()
def minD_0_19(L):
    A_temp = {}; A = []; D = [] 
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [1,2]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        if s == 1:
            A.append(list( tuple(d_temp[:s]+d_temp[:s]+d_temp[s:]) for d_temp in A_temp[s] ))
        else:
            A.append(list(set( list( tuple(d_temp[:2] + [max(d_temp[i] for i in range(2,4))]+d_temp[2:] ) for d_temp in A_temp[s] ) + list( tuple(d_temp[:3] + [max(d_temp[i] for i in range(2,4))]+d_temp[3:] ) for d_temp in A_temp[s] ) + list( tuple(d_temp + [max(d_temp[i] for i in range(2,4))] ) for d_temp in A_temp[s] ) )))
    Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_ite(A)) )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[19].graph6_string())] = minD_0_19(G5[19].adjacency_matrix())
DAS[str(G5[19].graph6_string())] = []
for d in mD[str(G5[19].graph6_string())]:
    L = diagonal_matrix(d) - G5[19].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[19].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[19].graph6_string())]))
print(len(DAS[str(G5[19].graph6_string())]))
print(s1-s0)

s0 = time.time()
def minD_0_20(L):
    A_temp = {}; A = []; D = []
    n = L.nrows() #size of L
    canonical_as = list( sum(L.row(i)) for i in range(n) )
    D.append(canonical_as)
    for s in [0]:
        L_s = reduced_matrix(L,s)
        A_temp[s] = minD_0(L_s)  #step(1)
        A.append(list( tuple([max(d_temp[i] for i in range(4))]+d_temp) for d_temp in A_temp[s] ) + list( tuple(d_temp[:1] + [max(d_temp[i] for i in range(4))]+d_temp[1:] ) for d_temp in A_temp[s] )+ list( tuple(d_temp[:2] + [max(d_temp[i] for i in range(4))]+d_temp[2:] ) for d_temp in A_temp[s] ) + list( tuple(d_temp[:3] + [max(d_temp[i] for i in range(4))]+d_temp[3:] ) for d_temp in A_temp[s] ) + list( tuple(d_temp + [max(d_temp[i] for i in range(4))] ) for d_temp in A_temp[s] ) )
    #Alpha = set( map(lambda vectors: up_vector_tuple(list(vectors),n), list_vectors_fun(A)) )
    Alpha = set( element for element in A[0] )
    Alpha = list( list(element) for element in Alpha)
    search_neg = []
    for d in minimals_iterator(Alpha): #...(3) "and" (4)
        L_d = diagonal_matrix(d) - L
        S = list(q for q in range(n) if reduced_matrix(L_d,q).det() == 0)
        [D,search_neg] = Find(L,d,S,D,Alpha,search_neg) #steps (6) to (14)
    return D #step (15)
mD[str(G5[20].graph6_string())] = minD_0_20(G5[20].adjacency_matrix())
DAS[str(G5[20].graph6_string())] = []
for d in mD[str(G5[20].graph6_string())]:
    L = diagonal_matrix(d) - G5[20].adjacency_matrix()
    if L.det() == 0:
        DAS[str(G5[20].graph6_string())].append(d)
s1 = time.time()
print(len(mD[str(G5[20].graph6_string())]))
print(len(DAS[str(G5[20].graph6_string())]))
print(s1-s0)

###The following lines check that we indeed found d-arith. Structures.
R.<x1,x2,x3,x4,x5,x6> = ZZ[]
for g in G5:
    for ea in DAS[str(g.graph6_string())]:
        eax = [ea[0]+x1,ea[1]+x2,ea[2]+x3,ea[3]+x4,ea[4]+x5]
        f = (diagonal_matrix(eax)-g.adjacency_matrix()).det()
        #print(f.coefficients(),len(f.coefficients()))
        if len(f.coefficients())!=31:
            print('nop',len(f.coefficients()),ea)
            break
        elif not T([1]*len(f.coefficients())) <= T(f.coefficients()):
            print('nop2',f.coefficients())
            break
###Next we compute $\min \mathcal{D}_{\geq 0}$ for all disconnected graphs with 5 vertices            
H5 = list(graphs(5))
F5 = [] #disconnected graphs with 5 vertices
for i in range(len(H5)):
    if not H5[i].is_connected():
        F5.append(H5[i])
print('Arithmetical structures for disconnected simple graphs with 5 vertices:')
for i in range(len(F5)):
    g = F5[i]
    #uncomment next line if you want to see a drawing of the graph.
    #g.show()
    A = g.adjacency_matrix()
    #uncomment next line if you want to see the adjacency matrix.
    #print(A)
    t0=time.time()
    mD[str(g.graph6_string())] = minD_0(A)
    t1=time.time()
    #uncomment next line to know how many vectors minD_0 has and the time that took to compute them.
    print('Length of minD_0 for ',str(g.graph6_string()),' is:' ,len(mD[str(g.graph6_string())]),' computed in ',(t1-t0),' seconds')
    #uncomment next line if you want to see a list of the d-arithmetical structures.
    #print(DAS[str(g.graph6_string())])
    #uncomment next line if you want to see a list of vectors in minD_0.
    #print(mD[str(g.graph6_string())])